In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from pylab import rcParams

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score

from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean

from imblearn.pipeline import Pipeline

from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.over_sampling import ADASYN

from sklearn.preprocessing import LabelEncoder

from numpy.random import seed
seed(7)

from sklearn.model_selection import train_test_split

SEED = 123 #used to help randomly select the data points
DATA_SPLIT_PCT = 0.2

rcParams['figure.figsize'] = 8, 6
LABELS = ["Normal","Break"]

In [2]:
df = pd.read_csv("bearing_dataset_combined_5%_augmented_stage2.csv") 
df.head(n=5)  # visualize the data.

,Unnamed: 0.1,Unnamed: 0,DE,FE,status,lag_DE,lag_FE,rolling_mean_DE,rolling_mean_FE,DE_pct_change_5,...,cnv_DE,cnv_FE,pool_DE,pool_FE,tw_DE,tw_FE,drift_DE,drift_FE,trend_DE,trend_FE
0,0,0,0.014603,0.192920,0,NaN,NaN,NaN,NaN,NaN,...,0.034047,0.176754,0.034526,0.178640,0.014603,0.192920,0.014603,0.192920,NaN,NaN
1,1,1,0.054449,0.164360,0,0.014603,0.192920,NaN,NaN,NaN,...,0.081158,0.128719,0.034526,0.178640,0.015371,0.087747,0.054458,0.164325,0.057788,0.153113
2,2,2,0.107650,0.090811,0,0.054449,0.164360,NaN,NaN,NaN,...,0.119790,0.092518,0.120685,0.088653,0.016150,0.088413,0.107667,0.090741,0.100867,0.108119
3,3,3,0.133720,0.086496,0,0.107650,0.090811,NaN,NaN,NaN,...,0.122511,0.090060,0.120685,0.088653,0.016941,0.013069,0.133745,0.086392,0.121935,0.090760
4,4,4,0.112650,0.099235,0,0.133720,0.086496,NaN,NaN,NaN,...,0.099459,0.096357,0.097527,0.096153,0.017743,0.085784,0.112684,0.099096,0.110356,0.094509


In [3]:
df['status'].value_counts()[1]

1000

In [4]:
df['status'].value_counts()[0]

19000

In [5]:
X = df.drop(['status'], axis=1)

y = df['status']

In [6]:
le = LabelEncoder()
y = le.fit_transform(y)
y

array([0, 0, 0, ..., 1, 1, 1])

In [7]:
X=X.fillna(0)
X

,Unnamed: 0.1,Unnamed: 0,DE,FE,lag_DE,lag_FE,rolling_mean_DE,rolling_mean_FE,DE_pct_change_5,FE_pct_change_5,...,cnv_DE,cnv_FE,pool_DE,pool_FE,tw_DE,tw_FE,drift_DE,drift_FE,trend_DE,trend_FE
0,0,0,0.014603,0.192920,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.034047,0.176754,0.034526,0.178640,0.014603,0.192920,0.014603,0.192920,0.000000,0.000000
1,1,1,0.054449,0.164360,0.014603,0.192920,0.000000,0.000000,0.000000,0.000000,...,0.081158,0.128719,0.034526,0.178640,0.015371,0.087747,0.054458,0.164325,0.057788,0.153113
2,2,2,0.107650,0.090811,0.054449,0.164360,0.000000,0.000000,0.000000,0.000000,...,0.119790,0.092518,0.120685,0.088653,0.016150,0.088413,0.107667,0.090741,0.100867,0.108119
3,3,3,0.133720,0.086496,0.107650,0.090811,0.000000,0.000000,0.000000,0.000000,...,0.122511,0.090060,0.120685,0.088653,0.016941,0.013069,0.133745,0.086392,0.121935,0.090760
4,4,4,0.112650,0.099235,0.133720,0.086496,0.000000,0.000000,0.000000,0.000000,...,0.099459,0.096357,0.097527,0.096153,0.017743,0.085784,0.112684,0.099096,0.110356,0.094509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,19995,-0.044435,-0.108480,-0.094711,-0.053007,-0.069886,-0.017405,0.601492,21.956301,...,-0.030059,-0.124748,-0.069573,-0.080743,-0.106573,-0.084597,0.077516,-0.257005,-0.049650,-0.102829
19996,19996,19996,-0.015020,-0.141350,-0.044435,-0.108480,-0.062137,-0.044202,-0.648786,9.749867,...,-0.024875,-0.126646,-0.023990,-0.128410,-0.106585,-0.077823,0.106952,-0.289857,-0.026859,-0.126662
19997,19997,19997,-0.032961,-0.115470,-0.015020,-0.141350,-0.062882,-0.060022,-0.644947,-18.030973,...,-0.057377,-0.100678,-0.023990,-0.128410,-0.106593,-0.084139,0.089031,-0.263960,-0.041410,-0.112077
19998,19998,19998,-0.084698,-0.076018,-0.032961,-0.115470,-0.068873,-0.069004,-0.278859,-17.818142,...,-0.094879,-0.088275,-0.095649,-0.086393,-0.106598,-0.090454,0.037315,-0.224490,-0.077239,-0.091069


In [9]:
# Feature scaling

In [8]:
X=X.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)
X

,DE,FE,lag_DE,lag_FE,rolling_mean_DE,rolling_mean_FE,DE_pct_change_5,FE_pct_change_5,expanding_mean_DE,expanding_mean_FE,cnv_DE,cnv_FE,pool_DE,pool_FE,tw_DE,tw_FE,drift_DE,drift_FE,trend_DE,trend_FE
0,0.014603,0.192920,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034047,0.176754,0.034526,0.178640,0.014603,0.192920,0.014603,0.192920,0.000000,0.000000
1,0.054449,0.164360,0.014603,0.192920,0.000000,0.000000,0.000000,0.000000,0.034526,0.178640,0.081158,0.128719,0.034526,0.178640,0.015371,0.087747,0.054458,0.164325,0.057788,0.153113
2,0.107650,0.090811,0.054449,0.164360,0.000000,0.000000,0.000000,0.000000,0.058901,0.149364,0.119790,0.092518,0.120685,0.088653,0.016150,0.088413,0.107667,0.090741,0.100867,0.108119
3,0.133720,0.086496,0.107650,0.090811,0.000000,0.000000,0.000000,0.000000,0.077605,0.133647,0.122511,0.090060,0.120685,0.088653,0.016941,0.013069,0.133745,0.086392,0.121935,0.090760
4,0.112650,0.099235,0.133720,0.086496,0.000000,0.000000,0.000000,0.000000,0.084614,0.126764,0.099459,0.096357,0.097527,0.096153,0.017743,0.085784,0.112684,0.099096,0.110356,0.094509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-0.044435,-0.108480,-0.094711,-0.053007,-0.069886,-0.017405,0.601492,21.956301,0.012952,0.032732,-0.030059,-0.124748,-0.069573,-0.080743,-0.106573,-0.084597,0.077516,-0.257005,-0.049650,-0.102829
19996,-0.015020,-0.141350,-0.044435,-0.108480,-0.062137,-0.044202,-0.648786,9.749867,0.012951,0.032724,-0.024875,-0.126646,-0.023990,-0.128410,-0.106585,-0.077823,0.106952,-0.289857,-0.026859,-0.126662
19997,-0.032961,-0.115470,-0.015020,-0.141350,-0.062882,-0.060022,-0.644947,-18.030973,0.012948,0.032716,-0.057377,-0.100678,-0.023990,-0.128410,-0.106593,-0.084139,0.089031,-0.263960,-0.041410,-0.112077
19998,-0.084698,-0.076018,-0.032961,-0.115470,-0.068873,-0.069004,-0.278859,-17.818142,0.012943,0.032711,-0.094879,-0.088275,-0.095649,-0.086393,-0.106598,-0.090454,0.037315,-0.224490,-0.077239,-0.091069


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape, X_test.shape

((16000, 20), (4000, 20))

In [10]:
X_train

,DE,FE,lag_DE,lag_FE,rolling_mean_DE,rolling_mean_FE,DE_pct_change_5,FE_pct_change_5,expanding_mean_DE,expanding_mean_FE,cnv_DE,cnv_FE,pool_DE,pool_FE,tw_DE,tw_FE,drift_DE,drift_FE,trend_DE,trend_FE
17815,0.080943,-0.074991,0.060707,-0.058349,0.032127,-0.023803,1.984624,2.230143,0.012909,0.032717,0.071381,-0.056206,0.070825,-0.066670,0.035105,0.117334,0.146033,-0.227865,0.071712,-0.062972
18370,0.043809,0.006986,0.053197,-0.045816,-0.012875,-0.019812,-1.763635,-2.888892,0.012949,0.032717,0.022324,0.038180,0.022322,0.037906,0.048651,-0.006120,0.124915,-0.150160,0.035412,0.009246
1379,0.050485,-0.074785,0.069052,-0.125330,0.047475,-0.089667,-0.086790,-0.087721,0.012240,0.031837,0.027574,-0.054193,0.059769,-0.100057,0.056682,-0.052483,0.040218,-0.109450,0.044227,-0.076635
14763,-0.037134,0.116290,0.019401,0.053418,-0.000119,0.022659,0.092048,-48.166903,0.012642,0.032647,-0.047196,0.127384,-0.008866,0.084854,0.044407,0.046808,-0.049545,0.012655,-0.028476,0.106272
7346,0.169810,0.141970,0.178780,0.065951,0.074177,0.080744,-12.628433,0.303793,0.012589,0.032363,0.155674,0.151785,0.153225,0.153580,-0.072305,0.058702,0.052523,0.078567,0.163760,0.128770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123,0.026703,-0.025065,0.001878,0.008013,-0.046462,0.044789,-1.359554,-1.580948,0.012406,0.032673,0.020766,-0.038013,0.014290,-0.008526,0.020585,0.077333,0.000591,-0.112198,0.018150,-0.022240
19648,-0.017315,0.039036,-0.025868,0.024038,-0.013620,0.093423,-1.468923,-0.680687,0.012964,0.032744,-0.040999,0.061933,-0.038281,0.061533,-0.047462,0.110209,0.097085,-0.114537,-0.029936,0.046535
9845,-0.027329,-0.034311,-0.095963,-0.007807,-0.052065,0.021338,-2.180212,-2.776575,0.012129,0.032408,-0.000142,-0.013487,-0.061646,-0.021059,-0.057567,-0.065307,-0.057892,-0.125794,-0.030249,-0.017669
10799,-0.021487,-0.029791,-0.023574,-0.064307,0.028043,-0.006076,-1.183933,-3.636372,0.012398,0.032583,-0.039515,0.013932,-0.022530,-0.047049,0.075680,-0.056889,-0.039354,-0.122444,-0.029206,-0.016385


In [11]:
# Feature scaling
# Fill missing values with a constant
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

X_train = X_train.replace([np.inf, -np.inf], 0) 
X_test = X_test.replace([np.inf, -np.inf], 0) 

cols = X_train.columns

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])


In [12]:
scaler = StandardScaler()


# Feature scaling
# Fill missing values with a constant
X = X.fillna(0)

X = X.replace([np.inf, -np.inf], 0) 

cols = X.columns

scaler = StandardScaler()
X = scaler.fit_transform(X)

X = pd.DataFrame(X, columns=[cols])



In [13]:
# define model
model = XGBClassifier()
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

Mean ROC AUC: 0.99992


In [14]:
model = XGBClassifier()

clf_0 = model.fit(X_train, y_train)


In [15]:
pred_y_0 = clf_0.predict(X_test)

print(accuracy_score(pred_y_0, y_test))



0.9965


In [16]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_0, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_0)

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      3802
     class 1       0.97      0.95      0.96       198

    accuracy                           1.00      4000
   macro avg       0.99      0.98      0.98      4000
weighted avg       1.00      1.00      1.00      4000



array([[3797,    5],
       [   9,  189]])

# Weighted XGBoost for Class Imbalance


In [17]:
# define model
#scale_pos_weight=total_negative_examples / total_positive_examples
#total number of examples in the majority class / total number of examples in the minority class.
#19900/100=199
model = XGBClassifier(scale_pos_weight=199)

# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

Mean ROC AUC: 0.99991


In [18]:
clf_1 = model.fit(X_train, y_train)
pred_y_1 = clf_1.predict(X_test)

print(accuracy_score(pred_y_1, y_test))

0.9955


In [19]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_1, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_1)

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      3802
     class 1       0.92      0.99      0.96       198

    accuracy                           1.00      4000
   macro avg       0.96      1.00      0.98      4000
weighted avg       1.00      1.00      1.00      4000



array([[3785,   17],
       [   1,  197]])

# Tune with GridSearch CV

In [20]:
# define grid
weights = [1, 10, 25, 50, 75, 99, 100, 1000]
param_grid = dict(scale_pos_weight=weights)

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')

In [ ]:
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [23]:
# define model
model = XGBClassifier(scale_pos_weight=30)

# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % np.mean(scores))


Mean ROC AUC: 0.99994


In [24]:
clf_2 = model.fit(X_train, y_train)
pred_y_2 = clf_2.predict(X_test)

print(accuracy_score(pred_y_2, y_test))



0.99575


In [25]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_2, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_2)

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      3802
     class 1       0.92      0.99      0.96       198

    accuracy                           1.00      4000
   macro avg       0.96      1.00      0.98      4000
weighted avg       1.00      1.00      1.00      4000



array([[3786,   16],
       [   1,  197]])